In [8]:
import json
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [9]:
options = Options()
options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})

url = "https://www.oddsportal.com/football/belgium/jupiler-pro-league/"
driver = webdriver.Chrome(options=options)
driver.get(url)
performance_logs = driver.get_log("performance")
driver.quit()


In [10]:
filtered_logs = []
filter_list = ["Network.request", "Network.response"]

for entry in performance_logs:
    networktraffic = json.loads(entry["message"])["message"]
    if any(keyword in networktraffic["method"] for keyword in filter_list):
        filtered_logs.append(networktraffic)

In [11]:
urls = []
for log in filtered_logs:
    try:
        if(log["params"]["type"] == 'XHR'):
            url = log["params"]["request"]["url"]
            urls.append(url)
    except:
        pass

filter_list = [".svg", ".png", "/js/", ".js", ".css", ".ico"]
filtered_urls = [url for url in urls if not any(filter_str in url for filter_str in filter_list)]

for url in filtered_urls:
    print(url)


https://content.livesportmedia.eu/delivery?zones=3991,3989,3990,5989,9977&clientType=PC&no_flash=1&programmatic=false
https://geolocation.onetrust.com/cookieconsentpub/v1/geo/location
https://www.oddsportal.com/ajax-all-events/topEvents
https://www.oddsportal.com/ajax-getCount/MyBookmarks/
https://www.oddsportal.com/ajax-getCount/MyCoupon/
https://www.oddsportal.com/ajax-sport-country-tournament_/1/nZN1aXIj/X360488X0X0X0X0X0X0X0X0X0X0X0X0X134217729X0X1048578X0X0X1024X18464X131072X256/1/?_=1724185892


# Finding the request(s) needed

In [12]:
import re

def check_url_contents(url, words=None, regex_patterns: list=None):
    headers = { 
           "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    response = requests.get(url, headers=headers)
    
    results = {'words': {}, 'regex_patterns': {}}
    
    if response.status_code == 200:
        content = response.text
        
        if words:
            for word in words:
                count = content.lower().count(word.lower())
                results['words'][word] = count
                
        if regex_patterns:
            for regex in regex_patterns:
                pattern = re.compile(regex)
                matches = pattern.findall(content)
                results['regex_patterns'][regex] = len(matches)
    
    return results

In [13]:
def rank_urls(urls, words=None, regex_patterns=None):
    url_scores = {}
    for url in urls:
        results = check_url_contents(url, words,regex_patterns)
        score=0
        if words:
            score+= sum(results['words'].values())
        if regex_patterns:
            score+= sum(results['regex_patterns'].values())
        url_scores[url] = score
        
    sorted_urls = sorted(url_scores.items(), key=lambda x:x[1], reverse=True)
    
    return sorted_urls
        

In [14]:

result = rank_urls(filtered_urls, ['bet', 'odds', 'avg', 'average', '2.01'])

for index, entry in enumerate(result, start=1):
        if entry[1] < 1:
            break
        print(f"{index}: {entry[0]}")

1: https://www.oddsportal.com/ajax-sport-country-tournament_/1/nZN1aXIj/X360488X0X0X0X0X0X0X0X0X0X0X0X0X134217729X0X1048578X0X0X1024X18464X131072X256/1/?_=1724185892
